# CSE6242 - HW3 - Q1

Pyspark Imports

In [1]:
### DO NOT MODIFY THIS CELL ###
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp, round, coalesce

Initialize PySpark Context

In [2]:
### DO NOT MODIFY THIS CELL ###
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/24 02:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Define function for loading data

In [3]:
### DO NOT MODIFY THIS CELL ###
def load_data():
    df = sqlContext.read.option("header",True) \
     .csv("yellow_tripdata_2019-01_short.csv")
    return df

### Q1.a

Perform data casting to clean incoming dataset

In [4]:
def clean_data(df):
    '''
    input: df a dataframe
    output: df a dataframe with the all the original columns
    '''
    
    # START YOUR CODE HERE ---------
    
    column_to_dtype = {
        "passenger_count": pyspark.sql.types.IntegerType(),
        "total_amount": pyspark.sql.types.FloatType(),
        "tip_amount": pyspark.sql.types.FloatType(),
        "trip_distance": pyspark.sql.types.FloatType(),
        "fare_amount": pyspark.sql.types.FloatType(),
        "tpep_pickup_datetime": pyspark.sql.types.TimestampType(),
        "tpep_dropoff_datetime": pyspark.sql.types.TimestampType()
    }
    
    for name, col_type in column_to_dtype.items():
        df = df.withColumn(name, col(name).cast(col_type))

    # END YOUR CODE HERE -----------
    
    return df

### Q1.b

Find rate per person for based on how many passengers travel between pickup and dropoff locations. 

In [13]:
def common_pair(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - PULocationID
            - DOLocationID
            - passenger_count
            - per_person_rate
            
    per_person_rate is the total_amount per person for a given pair.
    
    '''
    
    # START YOUR CODE HERE ---------

    df.createOrReplaceTempView("your_table_name")

    result = sqlContext.sql("""
        SELECT
            PULocationID,
            DOLocationID,
            SUM(passenger_count) AS passenger_count,
            SUM(total_amount) / SUM(passenger_count) AS per_person_rate
        FROM your_table_name
        WHERE PULocationID != DOLocationID
        GROUP BY PULocationID, DOLocationID
        ORDER BY passenger_count DESC, per_person_rate DESC
        LIMIT 10
    """)
    df = result

    # END YOUR CODE HERE -----------
    
    return df

### Q1.c

Find trips which trip distances generate the highest tip percentage.

In [43]:
def distance_with_most_tip(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - trip_distance
            - tip_percent
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    
    # START YOUR CODE HERE ---------

    df.createOrReplaceTempView("your_table_name")

    result = sqlContext.sql("""
        SELECT
            ROUND(trip_distance) AS trip_distance,
            AVG((tip_amount * 100) / fare_amount) AS trip_percent
        FROM your_table_name
        WHERE fare_amount > 2.0 AND trip_distance > 0
        GROUP BY ROUND(trip_distance)
        ORDER BY trip_percent DESC
        LIMIT 15
    """)
    result = result.orderBy("trip_percent")
    df = result
    
    # END YOUR CODE HERE -----------
    
    return df

### Q1.d

Determine the average speed at different times of day.

In [25]:
def time_with_most_traffic(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - time_of_day
            - am_avg_speed
            - pm_avg_speed
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    
    # START YOUR CODE HERE ---------

    df.createOrReplaceTempView("your_table_name")

    result = sqlContext.sql("""
        SELECT
            (CAST(HOUR(tpep_pickup_datetime) AS INT) % 12) AS time_of_day,
            AVG(CASE WHEN HOUR(tpep_pickup_datetime) < 12 THEN trip_distance / (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) ELSE NULL END) AS am_avg_speed,
            AVG(CASE WHEN HOUR(tpep_pickup_datetime) >= 12 THEN trip_distance / (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) ELSE NULL END) AS pm_avg_speed
        FROM your_table_name
        WHERE UNIX_TIMESTAMP(tpep_dropoff_datetime) > UNIX_TIMESTAMP(tpep_pickup_datetime)  -- Ensure trip time is positive
        GROUP BY (CAST(HOUR(tpep_pickup_datetime) AS INT) % 12)
        ORDER BY (CAST(HOUR(tpep_pickup_datetime) AS INT) % 12)
    """)

    df = result
    # END YOUR CODE HERE -----------
    
    return df

### The below cells are for you to investigate your solutions and will not be graded
## Ensure they are commented out prior to submitting to Gradescope to avoid errors

In [16]:
# df = load_data()
# df = clean_data(df)

In [17]:
# display(df)

DataFrame[VendorID: string, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: int, trip_distance: float, RatecodeID: string, store_and_fwd_flag: string, PULocationID: string, DOLocationID: string, payment_type: string, fare_amount: float, extra: string, mta_tax: string, tip_amount: float, tolls_amount: string, improvement_surcharge: string, total_amount: float, congestion_surcharge: string]

In [18]:
# common_pair(df).show()

+------------+------------+---------------+------------------+
|PULocationID|DOLocationID|passenger_count|   per_person_rate|
+------------+------------+---------------+------------------+
|         239|         238|             62|  4.26274198870505|
|         237|         236|             60| 4.482500068346659|
|         263|         141|             52|3.4190384974846473|
|         161|         236|             42| 5.368571440378825|
|         148|          79|             42| 4.711904752822149|
|         142|         238|             39|  5.05487182812813|
|         141|         236|             37| 4.355675723101641|
|         239|         143|             37| 4.252162224537617|
|         239|         142|             35| 3.817714350564139|
|          79|         170|             34| 6.394705884596881|
+------------+------------+---------------+------------------+



In [44]:
# distance_with_most_tip(df).show()

+-------------+------------------+
|trip_distance|      trip_percent|
+-------------+------------------+
|         18.0|11.405847876237262|
|         10.0|11.469710555853517|
|         23.0|11.666666666666666|
|          8.0|11.935883845460518|
|          6.0|13.301329030626446|
|          4.0|13.548640690629924|
|          9.0| 13.56675764510684|
|         19.0|14.024168214294264|
|         21.0|14.318693182631236|
|          3.0|14.467840950136962|
|          5.0|14.791297045921036|
|         17.0| 15.22190379132884|
|          2.0|15.316307008502697|
|          0.0|15.421387863221868|
|          1.0|16.948128912014667|
+-------------+------------------+



In [26]:
# time_with_most_traffic(df).show()

+-----------+--------------------+--------------------+
|time_of_day|        am_avg_speed|        pm_avg_speed|
+-----------+--------------------+--------------------+
|          0|0.003382123058407...|                NULL|
|          1|0.003020360777560...|0.001246246221187...|
|          3|                NULL|                 0.0|
|          4|                NULL|                 0.0|
|          5|                NULL|5.169266426901604E-4|
|          6|                NULL|0.002774957741846557|
|          7|                NULL|5.115362636227142E-5|
|          8|                NULL|0.001787106662091...|
|         10|                NULL|0.003181326500429...|
|         11|                NULL|0.003692714290889143|
+-----------+--------------------+--------------------+

